## import module

In [103]:
import numpy as np
from math import copysign
import matplotlib.pyplot as plt
from tqdm import tqdm
from missile_gym import MissileGym
from aero_info import *
import pickle
from matplotlib.pylab import subplot2grid
import ipywidgets
from ipywidgets import interact

In [104]:
rc = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]

## analasys mass

In [105]:
# длины отсеков
L_1, L_2, L_3, L_4, L_5 = 0.66693, 0.19608, 0.17479, 1.45209, 0.21012

In [106]:
# массы отсеков
Q1, Q2, Q3, Q5 = 28.5, 22, 11, 8.5
Q_воспл = 5

In [107]:
# из расчета РДТТ
w_marsh = 59.6
α_д = 0.255
Q4 = 74.71285
Q4_пуст = Q4 - w_marsh

In [108]:
L_korm = 41 / 1e3
d = 0.200
d_korm = 0.19304

L_korp = L_1 + L_2 + L_3 + L_4 + L_5
L_cil = (L_3 + L_4 + L_5 - L_korm)
L_nos = L_1 + L_2
L_korp, L_nos, L_cil

(2.70001, 0.86301, 1.7960000000000003)

In [109]:
# масса ракеты
Q1 + Q2 + Q3 + Q4 + Q_воспл + Q5

149.71285

In [110]:
# масса ракеты пустая
Q1 + Q2 + Q3 + Q4_пуст + Q_воспл + Q5

90.11285000000001

In [111]:
# центры масс отсеков
x_ct1 = L_1 * 4/7
x_ct2 = L_1 + L_2/2
x_ct3 = L_1 + L_2 + L_3 * 3/7
x_ct4 = L_1 + L_2 + L_3 + L_4 * 3/7
x_ct5 = L_1 + L_2 + L_3 + L_4 + L_5/3
x_ct1, x_ct2, x_ct3, x_ct4, x_ct5

(0.38110285714285713, 0.76497, 0.9379200000000001, 1.6601242857142857, 2.55993)

In [112]:
# центры масс ракеты в характерных точках
x_ct_0 = (Q1 * x_ct1 + Q2 * x_ct2 + Q3 * x_ct3 + Q4 * x_ct4 + Q5 * x_ct5) / (Q1 + Q2 + Q3 + Q4 + Q5)
x_ct_marsh = (Q1 * x_ct1 + Q2 * x_ct2 + Q3 * x_ct3 + Q4_пуст * x_ct4 + Q5 * x_ct5) / (Q1 + Q2 + Q3 + Q4_пуст + Q5)
x_ct_0, x_ct_marsh

(1.2701008456989133, 0.9969881838045439)

In [113]:
x_ct1 - x_ct_0, x_ct2 - x_ct_0, x_ct3 - x_ct_0, x_ct4 - x_ct_0, x_ct5 - x_ct_0

(-0.8889979885560562,
 -0.5051308456989133,
 -0.33218084569891326,
 0.39002344001537237,
 1.2898291543010867)

In [114]:
x_ct1 - x_ct_marsh, x_ct2 - x_ct_marsh, x_ct3 - x_ct_marsh, x_ct4 - x_ct_marsh, x_ct5 - x_ct_marsh

(-0.6158853266616868,
 -0.23201818380454386,
 -0.05906818380454382,
 0.6631361019097418,
 1.562941816195456)

In [115]:
I1 = Q1 * (x_ct1 - x_ct_0) ** 2
I2 = Q2 * (x_ct2 - x_ct_0) ** 2
I3 = Q3 * (x_ct3 - x_ct_0) ** 2
I4 = Q4 * (x_ct4 - x_ct_0) ** 2
I5 = Q5 * (x_ct5 - x_ct_0) ** 2
I0 = I1 + I2 + I3 + I4 + I5
I0

54.85758371788877

In [116]:
I1пуст = Q1 * (x_ct1 - x_ct_marsh) ** 2
I2пуст = Q2 * (x_ct2 - x_ct_marsh) ** 2
I3пуст = Q3 * (x_ct3 - x_ct_marsh) ** 2
I4пуст = Q4_пуст * (x_ct4 - x_ct_marsh) ** 2
I5пуст = (Q5) * (x_ct5 - x_ct_marsh) ** 2
Iпуст = I1пуст + I2пуст + I3пуст + I4пуст + I5пуст
Iпуст

39.44272174744154

In [117]:
# разбежка центра масс
razbezhka = (x_ct_0 - x_ct_marsh) / L_korp * 100
razbezhka

10.115246309990313

### 2.2. Определение АД характеристик на траектории полета полученного профиля скорости

## function

In [118]:
with open('saves/opts.bin', 'rb') as f:
    opts = pickle.load(f)

In [119]:
with open('saves/data_hit.bin', 'rb') as f:
    res_hit = pickle.load(f)

In [120]:
def get_P(t):
    if t < opts['t']:
        return opts['P']
    else:
        return 0

In [121]:
def get_x_ct(t):
    dx_ct = abs(x_ct_0 - x_ct_marsh) / opts['t']
    if t < opts['t']:
        return x_ct_0 - dx_ct * t
    else:
        return x_ct_marsh

In [122]:
def aero_coef(state):
    """
    Ф-ция расчёта аэродинамических коэффициентов ракеты в состоянии state
    arguments: state {np.ndarray}             -- состояние ракеты:
                                                 [v,   x, y, Q,       alpha,   t]
                                                 [м/с, м, м, радианы, градусы, с]
    return: {dict}                            -- словарь с АД коэф-тами 
    """ 
    
    v, x, y, alpha, t = state
    Mach = v / table_atm(y, 4)
    nyu = table_atm(y, 6)
    x_ct = get_x_ct(t)
    Re_korp_f = v * L_korp / nyu
    Re_korp_t = table_4_5(Mach, Re_korp_f, class_korp, L_korp)
    
    # Коэф-т подъемной силы корпуса
    if Mach <= 1:
        Cy_alpha_nos = 2 / 57.3 * (1 + 0.27 * Mach**2)
    else:
        Cy_alpha_nos = Cy_alpha_nos_cil(2, Mach, lambd_nos, lambd_cil)
    Cy_alpha_korm = - 2 / 57.3 * (1 - nu_korm ** 2) * a
    Cy_alpha_korp = Cy_alpha_nos + Cy_alpha_korm  
    
    # Коэф-т подъемной силы оперения по углу атаки
    K_t_oper = table_3_21(Mach, lambd_nos)
    Cy_alpha_k_oper = Cy_alpha_iz_kr(Mach * np.sqrt(K_t_oper), lambd_oper, c_oper, tg_khi_05_oper)
    k_aa_oper = (1 + 0.41 * D_oper)**2 * ((1 + 3 * D_oper - 1 / nu_k_oper * D_oper * (1 - D_oper)) / (1 + D_oper)**2)
    K_aa_oper = 1 + 3 * D_oper - (D_oper * (1 - D_oper)) / nu_k_oper
    delta_ps = 0.093 / ((v*L1/nyu)**0.2) * L1 / d * (1 + 0.4*Mach + 0.147*Mach**2 - 0.006*Mach**3)
    khi_ps = (1 - 2 * D_oper**2 * delta_ps / (1 - D_oper**2)) * (1 - D_oper * (nu_k_oper - 1) / ((1 - D_oper) * (1 + nu_k_oper)) * delta_ps)
    khi_nos = 0.6 + 0.4 * (1 - np.exp(-0.5 * L1 / d))
    K_aa_oper_ = K_aa_oper * khi_ps * khi_nos * 0.87
    Cy_alpha_oper = Cy_alpha_k_oper * K_aa_oper_
    
    # Коэф-т подъемной силы стабилизаторов по углу атаки
    K_t_stab = table_3_21(Mach, lambd_nos)
    Cy_alpha_k_stab = Cy_alpha_iz_kr(Mach * np.sqrt(K_t_stab), lambd_stab, c_stab, tg_khi_05_stab)
    k_aa_stab = (1 + 0.41 * D_stab)**2 * ((1 + 3 * D_stab - 1 / nu_k_stab * D_stab * (1 - D_stab)) / (1 + D_stab)**2)
    K_aa_stab = 1 + 3 * D_stab - (D_stab * (1 - D_stab)) / nu_k_stab
    delta_ps = 0.093 / ((v*L1/nyu)**0.2) * L1 / d * (1 + 0.4*Mach + 0.147*Mach**2 - 0.006*Mach**3)
    khi_ps = (1 - 2 * D_stab**2 * delta_ps / (1 - D_stab**2)) * (1 - D_stab * (nu_k_stab - 1) / ((1 - D_stab) * (1 + nu_k_stab)) * delta_ps)
    khi_nos = 0.6 + 0.4 * (1 - np.exp(-0.5 * L1 / d))
    K_aa_stab_ = K_aa_stab * khi_ps * khi_nos * 0.87
    Cy_alpha_stab = Cy_alpha_k_stab * K_aa_stab_
    
    # Коэф-т подъемной силы оперения (рулей) по углу их отклонения
    K_delt_0_oper = k_aa_oper
    k_delt_0_oper = k_aa_oper ** 2 / K_aa_oper
    if Mach <= 1:
        k_shch = 0.825
    elif 1 < Mach <= 1.4:
        k_shch = 0.85 + 0.15 * (Mach - 1) / 0.4
    else:
        k_shch = 0.975
    n_eff = k_shch * np.cos(np.radians(khi_rul))
    Сy_delt_oper = Cy_alpha_k_oper * K_delt_0_oper * n_eff
    
    # Коэф-т подъемной силы ракеты
    Cy_alpha = Cy_alpha_korp * (S_mid / S_mid)  + Cy_alpha_oper * (S_oper / S_mid) * K_t_oper + Cy_alpha_stab * (S_stab / S_mid)
    
    # Сопротивление корпуса
    x_t = Re_korp_t * nyu / v
    x_t_ = x_t / L_korp
    Cx_f_ = table_4_2(Re_korp_f, x_t_) / 2
    nu_m = table_4_3(Mach, x_t_)
    nu_c = 1 + 1 / lambd_nos
    Cx_tr = Cx_f_ * (F_f / S_mid) * nu_m * nu_c
    
    if Mach >= 1.5:
        p_kon_ = (0.0016 + 0.002 / (Mach**2)) * betta_kon1**1.7
        Cx_nos = p_kon_ * (1 - (14 - 1.143 / (lambd_nos**2)) / (Mach + 18))
    else:
        Cx_nos = table_4_11(Mach, lambd_nos)

    Cx_korm = table_4_24(Mach, nu_korm, lambd_korm)
    
    if get_P(t) == 0:
        p_dno_ = table_p_dno_(Mach, oper=True)
        K_nu = table_k_nu(nu_korm, lambd_korm, Mach)
        Cx_dno = p_dno_ * K_nu * (S_dno / S_mid)
    else:
        Cx_dno = 0
    Cx_0_korp = Cx_tr + Cx_nos + Cx_korm + Cx_dno
    
    if Mach < 1:
        phi = -0.2
    else:
        phi = 0.7
    Cx_ind_korp = Cy_alpha_korp * alpha**2 * ((1 + phi) / 57.3)
    
    Cx_korp = Cx_0_korp + Cx_ind_korp
     
    # Сопротивление оперения
    Re_oper_f = v * b_a_oper / nyu
    Re_oper_t = table_4_5(Mach, Re_oper_f, class_oper, b_a_oper)
    x_t_oper = Re_oper_t / Re_oper_f
    C_f_oper = table_4_2(Re_oper_f, x_t_oper)
    nu_c_oper = table_4_28(x_t_oper, c_oper)
    Cx_oper_prof = C_f_oper * nu_c_oper
    
    if Mach < 1.1:
        Cx_oper_voln = table_4_30(Mach, nu_k_oper, lambd_oper, tg_khi_05_oper, c_oper)
    else:
        phi = table_4_32(Mach, tg_khi_05_oper)
        Cx_oper_voln = (table_4_30(Mach, nu_k_oper, lambd_oper, tg_khi_05_oper, c_oper)) * (1 + phi * (K_oper - 1))
        
    Cx_0_oper = Cx_oper_prof + Cx_oper_voln
    
    if Mach * np.cos(np.radians(khi_pk_oper)) > 1:
        Cx_ind_oper = (Cy_alpha_oper * alpha) * np.tan(np.radians(alpha))
    else:
        Cx_ind_oper = 0.38 * (Cy_alpha_oper * alpha)**2 / (lambd_oper - 0.8 * (Cy_alpha_oper * alpha) * (lambd_oper - 1)) * ((lambd_oper / np.cos(np.radians(khi_pk_oper)) + 4) / (lambd_oper + 4))
        
    Cx_oper = Cx_0_oper + Cx_ind_oper
    
    # Сопротивление стабилизаторов
    Re_stab_f = v * b_a_stab / nyu
    Re_stab_t = table_4_5(Mach, Re_stab_f, class_stab, b_a_stab)
    x_t_stab = Re_stab_t / Re_stab_f
    C_f_stab = table_4_2(Re_stab_f, x_t_stab)
    nu_c_stab = table_4_28(x_t_stab, c_stab)
    Cx_stab_prof = C_f_stab * nu_c_stab
    
    if Mach < 1.1:
        Cx_stab_voln = table_4_30(Mach, nu_k_stab, lambd_stab, tg_khi_05_stab, c_stab)
    else:
        phi = table_4_32(Mach, tg_khi_05_stab)
        Cx_stab_voln = (table_4_30(Mach, nu_k_stab, lambd_stab, tg_khi_05_stab, c_stab)) * (1 + phi * (K_stab - 1))
        
    Cx_0_stab = Cx_stab_prof + Cx_stab_voln
    
    if Mach * np.cos(np.radians(khi_pk_stab)) > 1:
        Cx_ind_stab = (Cy_alpha_stab * alpha) * np.tan(np.radians(alpha))
    else:
        Cx_ind_stab = 0.38 * (Cy_alpha_stab * alpha)**2 / (lambd_stab - 0.8 * (Cy_alpha_stab * alpha) * (lambd_stab - 1)) * ((lambd_stab / np.cos(np.radians(khi_pk_stab)) + 4) / (lambd_stab + 4))
        
    Cx_stab = Cx_0_stab + Cx_ind_stab
    
    Cx_0 = 1.05 * (Cx_0_korp * (S_mid / S_mid) + Cx_0_oper * K_t_oper * (S_oper / S_mid) + Cx_0_stab * K_t_stab * (S_stab / S_mid))
    Cx_ind = Cx_ind_korp * (S_mid / S_mid) + Cx_ind_oper * (S_oper / S_mid) * K_t_oper
    Cx = Cx_0 + Cx_ind + Cx_stab
        
    # Центр давления корпуса
    delta_x_f = F_iz_korp(Mach, lambd_nos, lambd_cil, L_nos)
    x_fa_nos_cil = L_nos - W_nos / S_mid + delta_x_f
    x_fa_korm = L_korp - 0.5 * L_korm   
    x_fa_korp = 1 / Cy_alpha_korp * (Cy_alpha_nos * x_fa_nos_cil + Cy_alpha_korm * x_fa_korm)
    
    # Фокус оперения по углу атаки
    x_f_iz_oper_ = F_iz_kr(Mach, lambd_k_oper, tg_khi_05_oper, nu_k_oper)
    x_f_iz_oper = x_b_a_oper + b_a_oper * x_f_iz_oper_
    f1 = table_5_11(D_oper, L_k_oper)
    x_f_delt_oper = x_f_iz_oper - tg_khi_05_oper * f1
    if Mach > 1:
        b__b_oper = b_b_oper / (np.pi / 2 * d * np.sqrt(Mach ** 2 - 1))
        L__hv_oper = L_hv_oper / (np.pi * d * np.sqrt(Mach ** 2 - 1))
        c_const_oper = (4 + 1 / nu_k_oper) * (1 + 8 * D_oper ** 2)
        F_1_oper = 1 - 1 / (c_const_oper * b__b_oper ** 2) * (1 - np.exp(-c_const_oper * b__b_oper ** 2))
        F_oper = 1 - np.sqrt(np.pi) / (2 * b__b_oper * np.sqrt(c_const_oper)) * (table_int_ver((b__b_oper + L__hv_oper) *\
                np.sqrt(2 * c_const_oper)) - table_int_ver(L__hv_oper * np.sqrt(2 * c_const_oper)))
        x_f_b_oper_ = x_f_iz_oper_ + 0.02 * lambd_oper * tg_khi_05_oper
        x_f_ind_oper = x_b_oper + b_b_oper * x_f_b_oper_ * F_oper * F_1_oper
        x_fa_oper = 1 / K_aa_oper * (x_f_iz_oper + (k_aa_oper - 1) * x_f_delt_oper + (K_aa_oper - k_aa_oper) * x_f_ind_oper)
    else:
        x_f_b_oper_ = x_f_iz_oper_ + 0.02 * lambd_oper * tg_khi_05_oper
        x_f_ind_oper = x_b_oper + b_b_oper * x_f_b_oper_
        x_fa_oper = 1 / K_aa_oper * (x_f_iz_oper + (k_aa_oper - 1) * x_f_delt_oper + (K_aa_oper - k_aa_oper) * x_f_ind_oper)
    
    # Фокус стабилизаторов по углу атаки
    x_f_iz_stab_ = F_iz_kr(Mach, lambd_k_stab, tg_khi_05_stab, nu_k_stab)
    x_f_iz_stab = x_b_a_stab + b_a_stab * x_f_iz_stab_
    f1 = table_5_11(D_stab, L_k_stab)
    x_f_delt_stab = x_f_iz_stab - tg_khi_05_stab * f1
    if Mach > 1:
        b__b_stab = b_b_stab / (np.pi / 2 * d * np.sqrt(Mach ** 2 - 1))
        L__hv_stab = L_hv_stab / (np.pi * d * np.sqrt(Mach ** 2 - 1))
        c_const_stab = (4 + 1 / nu_k_stab) * (1 + 8 * D_stab ** 2)
        F_1_stab = 1 - 1 / (c_const_stab * b__b_stab ** 2) * (1 - np.exp(-c_const_stab * b__b_stab ** 2))
        F_stab = 1 - np.sqrt(np.pi) / (2 * b__b_stab * np.sqrt(c_const_stab)) * (table_int_ver((b__b_stab + L__hv_stab) * np.sqrt(2 * c_const_stab)) - table_int_ver(L__hv_stab * np.sqrt(2 * c_const_stab)))
        x_f_b_stab_ = x_f_iz_stab_ + 0.02 * lambd_stab * tg_khi_05_stab
        x_f_ind_stab = x_b_stab + b_b_stab * x_f_b_stab_ * F_stab * F_1_stab
        x_fa_stab = 1 / K_aa_stab * (x_f_iz_stab + (k_aa_stab - 1) * x_f_delt_stab + (K_aa_stab - k_aa_stab) * x_f_ind_stab)
    else:
        x_f_b_stab_ = x_f_iz_stab_ + 0.02 * lambd_stab * tg_khi_05_stab
        x_f_ind_stab = x_b_stab + b_b_stab * x_f_b_stab_
        x_fa_stab = 1 / K_aa_stab * (x_f_iz_stab + (k_aa_stab - 1) * x_f_delt_stab + (K_aa_stab - k_aa_stab) * x_f_ind_stab)
        
    # Фокус оперения по углу отклонения
    x_fd_oper = 1 / K_delt_0_oper * (k_delt_0_oper * x_f_iz_oper + (K_delt_0_oper - k_delt_0_oper) * x_f_ind_oper)
    
    # Фокус ракеты
    x_fa = 1 / Cy_alpha * ((Cy_alpha_korp * (S_mid / S_mid) * x_fa_korp) + Cy_alpha_oper * (S_oper / S_mid) * x_fa_oper * K_t_oper + Cy_alpha_stab * (S_stab / S_mid) * x_fa_stab * K_t_stab)
    
    # Демпфирующие моменты АД поверхностей
    x_c_ob = L_korp * ((2 * (lambd_nos + lambd_cil)**2 - lambd_nos**2) / (4 * (lambd_nos + lambd_cil) * (lambd_nos + lambd_cil - 2 / 3 * lambd_nos)))
    m_z_wz_korp = - 2 * (1 - x_ct / L_korp + (x_ct / L_korp) ** 2 - x_c_ob / L_korp)

    x_ct_oper_ = (x_ct - x_b_a_oper) / b_a_oper
    mz_wz_cya_iz_kr = table_5_15(nu_oper, lambd_oper, tg_khi_05_oper, Mach)
    B1 = table_5_16(lambd_oper, tg_khi_05_oper, Mach)
    m_z_wz_oper = (mz_wz_cya_iz_kr - B1 * (1 / 2 - x_ct_oper_) - 57.3 * (1 / 2 - x_ct_oper_)**2) * K_aa_oper * Cy_alpha_k_oper

    x_ct_stab_ = (x_ct - x_b_a_stab) / b_a_stab
    mz_wz_cya_iz_stab = table_5_15(nu_stab, lambd_stab, tg_khi_05_stab, Mach)
    B1 = table_5_16(lambd_stab, tg_khi_05_stab, Mach)
    m_z_wz_stab = (mz_wz_cya_iz_stab - B1 * (1 / 2 - x_ct_stab_) - 57.3 * (1 / 2 - x_ct_stab_)**2) * K_aa_stab * Cy_alpha_k_stab
    
    m_z_wz = m_z_wz_korp * (S_mid / S_mid) * (L_korp / L_korp)**2 + m_z_wz_oper * (S_oper / S_mid) * (b_a_oper / L_korp)**2 * np.sqrt(K_t_oper) + m_z_wz_stab * (S_stab / S_mid) * (b_a_stab / L_korp)**2 * np.sqrt(K_t_stab)

    # Балансировочная зависимость
    M_z_delt = Сy_delt_oper * (x_ct - x_fd_oper) / L_korp
    M_z_alpha = Cy_alpha * (x_ct - x_fa) / L_korp
    ballans_relation = -(M_z_alpha / M_z_delt)

    
    # Запас статической устойчивости
    m_z_cy = (x_ct - x_fa) / L_korp
    
    
    return {
        'Cy_alpha': Cy_alpha,
        'Cy_alpha_korp': Cy_alpha_korp,
        'Cy_alpha_oper': Cy_alpha_oper,
        'Cy_alpha_stab': Cy_alpha_stab,
        'Cx': Cx,
        'Cx_0': Cx_0,
        'Cx_0_korp': Cx_0_korp,
        'Cx_0_oper': Cx_0_oper,
        'Cx_0_stab': Cx_0_stab,
        'Cx_ind': Cx_ind,
        'Cx_ind_korp': Cx_ind_korp,
        'Cx_ind_oper': Cx_ind_oper,
        'Cx_ind_stab': Cx_ind_stab,
        'x_fa': x_fa,
        'x_fa_korp': x_fa_korp,
        'x_fa_oper': x_fa_oper,
        'x_fa_stab': x_fa_stab,
        'x_fd_oper': x_fd_oper,
        'm_z_cy': m_z_cy,
        'm_z_wz': m_z_wz,
        'm_z_wz_korp': m_z_wz_korp,
        'm_z_wz_oper': m_z_wz_oper,
        'm_z_wz_stab': m_z_wz_stab,
        'ballans_relation': ballans_relation,
        'M_z_alpha': M_z_alpha,
        'M_z_delt': M_z_delt
    }

def foo_aero(solve):
    
    Cya = []
    Cy_alpha_korp = []
    Cy_alpha_oper = []
    Cy_alpha_stab = []
    Cx = []
    Cx_0 = []
    Cx_ind = []
    Cx_0_korp = []
    Cx_ind_korp = []
    Cx_0_oper = []
    Cx_ind_oper = []
    Cx_0_stab = []
    Cx_ind_stab = []
    m_z_cy = []
    m_z_wz = []
    m_z_wz_korp = []
    m_z_wz_oper = []
    m_z_wz_stab = []
    x_fa = []
    x_fa_korp = []
    x_fa_oper = []
    x_fa_stab = []
    x_fd_oper = []
    ballans_relation = []
    M_z_alpha = []
    M_z_delt = []
    t = []
    
    for r in range(len(solve['t'])):
        arg = [solve['missile']['v'][r], solve['missile']['x'][r], solve['missile']['y'][r], solve['missile']['alpha'][r], solve['t'][r]]
        res_aero = aero_coef(arg)
        
        Cya.append(res_aero['Cy_alpha'])
        Cy_alpha_korp.append(res_aero['Cy_alpha_korp'])
        Cy_alpha_oper.append(res_aero['Cy_alpha_oper'])
        Cy_alpha_stab.append(res_aero['Cy_alpha_stab'])
        
        Cx.append(res_aero['Cx'])
        Cx_0.append(res_aero['Cx_0'])
        Cx_ind.append(res_aero['Cx_ind'])
        Cx_0_korp.append(res_aero['Cx_0_korp'])
        Cx_ind_korp.append(res_aero['Cx_ind_korp'])
        Cx_0_oper.append(res_aero['Cx_0_oper'])
        Cx_ind_oper.append(res_aero['Cx_ind_oper'])
        Cx_0_stab.append(res_aero['Cx_0_stab'])
        Cx_ind_stab.append(res_aero['Cx_ind_stab'])
        
        m_z_cy.append(res_aero['m_z_cy'])
        m_z_wz.append(res_aero['m_z_wz'])
        m_z_wz_korp.append(res_aero['m_z_wz_korp'])
        m_z_wz_oper.append(res_aero['m_z_wz_oper'])
        m_z_wz_stab.append(res_aero['m_z_wz_stab'])
        
        x_fa.append(res_aero['x_fa'])
        x_fa_korp.append(res_aero['x_fa_korp'])
        x_fa_oper.append(res_aero['x_fa_oper'])
        x_fa_stab.append(res_aero['x_fa_stab'])
        x_fd_oper.append(res_aero['x_fd_oper'])
        
        ballans_relation.append(res_aero['ballans_relation'])
        M_z_alpha.append(res_aero['M_z_alpha'])
        M_z_delt.append(res_aero['M_z_delt'])
        
        t.append(solve['t'][r])
        
    return {
        'Cy_alpha': Cya,
        'Cy_alpha_korp': Cy_alpha_korp,
        'Cy_alpha_oper': Cy_alpha_oper,
        'Cy_alpha_stab': Cy_alpha_stab,
        'Cx': Cx,
        'Cx_0': Cx_0,
        'Cx_0_korp': Cx_0_korp,
        'Cx_0_oper': Cx_0_oper,
        'Cx_0_stab': Cx_0_stab,
        'Cx_ind': Cx_ind,
        'Cx_ind_korp': Cx_ind_korp,
        'Cx_ind_oper': Cx_ind_oper,
        'Cx_ind_stab': Cx_ind_stab,
        'x_fa': x_fa,
        'x_fa_korp': x_fa_korp,
        'x_fa_oper': x_fa_oper,
        'x_fa_stab': x_fa_stab,
        'x_fd_oper': x_fd_oper,
        'm_z_cy': m_z_cy,
        'm_z_wz': m_z_wz,
        'm_z_wz_korp': m_z_wz_korp,
        'm_z_wz_oper': m_z_wz_oper,
        'm_z_wz_stab': m_z_wz_stab,
        'ballans_relation': ballans_relation,
        'M_z_alpha': M_z_alpha,
        'M_z_delt': M_z_delt,
        't': t
    }

In [123]:
b_0_oper, S_oper, x_b_oper, L_oper, khi_pk_oper, c_oper = 0.110, 0.0494, 2.529, 0.380, 0, 0.03
b_0_stab, S_stab, x_b_stab, L_stab, khi_pk_stab, c_stab = 0.110, 0.03751, 2.410, 0.380, 49, 0.03
betta_kon1 = 15
F_f = 1.613752818061
W_nos = 0.017674
khi_rul = 0
class_korp = 7
class_oper = 5
class_stab = 5
a = 0.2

In [124]:
# Вычисление геометрии корпуса:
L_nos = L_1 + L_2
S_mid = np.pi * d**2 / 4
S_dno = np.pi * d_korm**2 / 4
    
lambd_korp = L_korp / d
lambd_nos = L_nos / d
lambd_cil = L_cil / d
lambd_korm = L_korm / d
nu_korm = d_korm / d  
    
# Вычисление геометрии рулей (оперения):
D_oper = d / L_oper
L_k_oper = L_oper - d
tg_khi_pk_oper = np.tan(np.radians(khi_pk_oper))
lambd_oper = L_oper**2 / S_oper
nu_oper = (S_oper / (L_oper * b_0_oper) - 0.5) ** (-1) / 2
b_k_oper = b_0_oper / nu_oper
b_a_oper = 4 / 3 * S_oper / L_oper * (1 - (nu_oper / (nu_oper + 1) ** 2))
b_b_oper = b_0_oper * (1 - (nu_oper - 1) / nu_oper * d / L_oper)
z_a_oper = L_oper / 6 * ((nu_oper + 2) / (nu_oper + 1))
S_k_oper = S_oper * (1 - ((nu_oper - 1) / (nu_oper + 1)) * d / L_oper) * (1 - d / L_oper)
nu_k_oper = nu_oper - d / L_oper * (nu_oper - 1)
lambd_k_oper = lambd_oper * ((1 - d / L_oper) / (1 - ((nu_oper - 1) / (nu_oper + 1) * d / L_oper)))
tg_khi_05_oper = tg_khi_pk_oper - 2 / lambd_oper * (nu_k_oper - 1) / (nu_k_oper + 1)
a_oper = 2/3 * b_b_oper
K_oper = 1 / (1 - a_oper / b_a_oper)
L_hv_oper = L_korp - x_b_oper - b_b_oper
L1 = x_b_oper + b_b_oper/2
if tg_khi_pk_oper == 0:
    x_b_a_oper = x_b_oper
else:
    x_b_a_oper = x_b_oper + (z_a_oper - d / 2) * tg_khi_pk_oper
    
# Вычисление геометрии крыльев (стабилизаторов):
D_stab = d / L_stab
L_k_stab = L_stab - d
tg_khi_pk_stab = np.tan(np.radians(khi_pk_stab))
lambd_stab = L_stab**2 / S_stab
nu_stab = (S_stab / (L_stab * b_0_stab) - 0.5) ** (-1) / 2
b_k_stab = b_0_stab / nu_stab
b_a_stab = 4 / 3 * S_stab / L_stab * (1 - (nu_stab / (nu_stab + 1) ** 2))
b_b_stab = b_0_stab * (1 - (nu_stab - 1) / nu_stab * d / L_stab)
z_a_stab = L_stab / 6 * ((nu_stab + 2) / (nu_stab + 1))
S_k_stab = S_stab * (1 - ((nu_stab - 1) / (nu_stab + 1)) * d / L_stab) * (1 - d / L_stab)
nu_k_stab = nu_stab - d / L_stab * (nu_stab - 1)
lambd_k_stab = lambd_stab * ((1 - d / L_stab) / (1 - ((nu_stab - 1) / (nu_stab + 1) * d / L_stab)))
tg_khi_05_stab = tg_khi_pk_stab - 2 / lambd_stab * (nu_k_stab - 1) / (nu_k_stab + 1)
a_stab = 2/3 * b_b_stab
K_stab = 1 / (1 - a_stab / b_a_stab)
L_hv_stab = L_korp - x_b_stab - b_b_stab
L1 = x_b_stab + b_b_stab/2
if tg_khi_pk_stab == 0:
    x_b_a_stab = x_b_stab
else:
    x_b_a_stab = x_b_stab + (z_a_stab - d / 2) * tg_khi_pk_stab

In [125]:
L_nos

0.86301

In [126]:
lambd_oper, nu_oper, b_k_oper, b_b_oper, S_k_oper

(2.923076923076923,
 0.7333333333333335,
 0.14999999999999997,
 0.13105263157894737,
 0.025294736842105262)

In [127]:
lambd_stab, nu_stab, b_k_stab, b_b_stab, S_k_stab

(3.8496400959744066,
 1.2582781456953644,
 0.08742105263157893,
 0.0981163434903047,
 0.01669836565096953)

In [128]:
lambd_korp, lambd_nos, lambd_cil, lambd_korm, nu_korm

(13.500049999999998, 4.31505, 8.98, 0.205, 0.9652)

In [129]:
S_k_stab / S_k_oper 

0.6601517839542678

In [130]:
res_aero = foo_aero(res_hit)

In [131]:
@interact(i=ipywidgets.IntSlider(description='tau', max=len(res_hit['t'])-1, step=1, value=len(res_hit['t'])-1))
def draw_aero_gym_hit(i):
    
    plt.figure(figsize=(16,12))

    ax00 = subplot2grid((8, 4), (0, 2), colspan=2, rowspan=2)
    ax01 = subplot2grid((8, 4), (0, 0), colspan=2, rowspan=2)
    ax02 = subplot2grid((8, 4), (2, 0), colspan=2, rowspan=2)
    ax03 = subplot2grid((8, 4), (4, 0), colspan=2, rowspan=2,sharex=ax02)
    ax04 = subplot2grid((8, 4), (2, 2), colspan=2, rowspan=2,sharex=ax02)
    ax05 = subplot2grid((8, 4), (4, 2), colspan=2, rowspan=2,sharex=ax02)
    ax06 = subplot2grid((8, 4), (6, 0), colspan=2, rowspan=2,sharex=ax02)
    ax07 = subplot2grid((8, 4), (6, 2), colspan=2, rowspan=2,sharex=ax02)
    
    ax00.plot(res_aero['t'][:i], res_hit['missile']['alpha'][:i], lw=3, ls=':', color='k', label='$alpha$')
    ax00.legend(loc='best', fontsize=14)
    ax00.tick_params(left=False, labelleft=False, right=True, labelright=True,  pad = 0, labelsize=12)
    ax00.grid(True)
    
    ax01.plot(res_aero['t'][:i], res_hit['missile']['v'][:i], lw=3, ls=':', color='k', label='скорость ракеты')
    ax01.tick_params(labelsize=12, pad = 0)
    ax01.legend(loc='best', fontsize=14)
    ax01.grid(True)
    
    ax02.plot(res_aero['t'][:i], res_aero['Cy_alpha'][:i], label='$C_{Y.ЛА}^{a}$', c='k')
    ax02.plot(res_aero['t'][:i], res_aero['Cy_alpha_korp'][:i], label='$C_{Y.корп}^{a}$', c='blue')
    ax02.plot(res_aero['t'][:i], res_aero['Cy_alpha_oper'][:i], label='$C_{Y.опер}^{a}$', c='darkorange')
    ax02.tick_params(labeltop=False, top=False, labelbottom=False, bottom=False, labelsize=12)
    ax02.legend(fontsize=14)
    ax02.grid(True)
    
    ax03.plot(res_aero['t'][:i], res_aero['Cx'][:i], label='$C_{X.ЛА}$', c='k')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_0'][:i], label=r'$C_{X0.ЛА}$', c='k', ls='-.')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_ind'][:i], label=r'$C_{Xind.ЛА}$', c='k', ls='--')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_0_korp'][:i], label=r'$C_{X0.корп}$', c='blue', ls='-.')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_ind_korp'][:i], label=r'$C_{Xind.корп}$', c='blue', ls='--')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_0_oper'][:i], label=r'$C_{X0.опер}$', c='darkorange', ls='-.')
    ax03.plot(res_aero['t'][:i], res_aero['Cx_ind_oper'][:i], label=r'$C_{Xind.опер}$', c='darkorange', ls='--')
    ax03.legend(loc='best', ncol=3, fontsize=14)
    ax03.tick_params(left=True, labelleft=True, labelbottom=False, bottom=False, labelsize=12)
    ax03.grid(True)
    
    ax04.plot(res_aero['t'][:i], res_aero['m_z_wz'][:i], label='$m_z^{wz}$', color='black', ls='-')
    ax04.plot(res_aero['t'][:i], res_aero['m_z_wz_korp'][:i], label='$m_{z.корп}^{wz}$', color='blue', ls='--')
    ax04.plot(res_aero['t'][:i], res_aero['m_z_wz_oper'][:i], label='$m_{z.опер}^{wz}$', color='darkorange', ls='-.')
    ax04.tick_params(labeltop=False, top=False, pad = 5, left=False, labelleft=False, right=True, labelright=True, labelbottom=False, bottom=False, labelsize=12)
    ax04.legend(fontsize=14)
    ax04.grid(True)
       
    ax05.plot(res_aero['t'][:i], res_aero['x_fa'][:i], color='k', label='$x_{fa}$')
    ax05.plot(res_aero['t'][:i], res_aero['x_fa_korp'][:i], color='blue', label='$x_{fa.korp}$')
    ax05.plot(res_aero['t'][:i], res_aero['x_fa_oper'][:i], color='darkorange', label='$x_{fa.oper}$')
    ax05.tick_params(left=False, labelleft=False, right=True, labelright=True, labelbottom=False, bottom=False, labelsize=12)
    ax05.legend(fontsize=14)
    ax05.grid(True)
    
    ax06.plot(res_aero['t'][:i], res_aero['m_z_cy'][:i], label='$m_z^{Cy}$', color='k')
    ax06.plot(res_aero['t'][:i], res_aero['M_z_delt'][:i], label='$M_{z}^δ$', ls='-.', color='k')
    ax06.plot(res_aero['t'][:i], res_aero['M_z_alpha'][:i], label='$M_{z}^α$', ls='--',color='k')
    ax06.tick_params(labelbottom=True, bottom=True, labelsize=12)
    ax06.legend(fontsize=14, loc='right')
    ax06.grid(True)
    
    ax07.plot(res_aero['t'][:i], res_aero['ballans_relation'][:i], label='$δ / α$', color='k')
    ax07.tick_params(left=False, labelleft=False, right=True, labelright=True, labelbottom=True, bottom=True, labelsize=12)
    ax07.legend(fontsize=14)
    ax07.grid(True)
    
#     plt.savefig('pictures/Аэродинамика ЛА.png', dpi=300, bbox_inches='tight')
    
    plt.show()

interactive(children=(IntSlider(value=242, description='tau', max=242), Output()), _dom_classes=('widget-inter…